<a href="https://colab.research.google.com/github/translatorswb/Language-Use-Data-Platform-examples/blob/main/Typoon_%26_Language%20example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Install dash (if necessary)
%pip install dash

In [ ]:
# @title Import libraries
import requests
import pandas as pd
import geopandas as gpd
import plotly.express as px
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State

In [ ]:
# @title Fetching typhoon data as geojson

#fetching geojson
url = "https://agora.ex.nii.ac.jp/digital-typhoon/geojson/wnp/202401.en.json"
response = requests.get(url)
typhoon_geojson = response.json()
typhoon_data = gpd.GeoDataFrame.from_features(typhoon_geojson['features'])

#setting coordinate reference system
typhoon_data.set_crs(epsg=4326, inplace=True)

#expanding points to areas by setting a buffer based on the 'class' of typhoon at each point (requires a different crs)
typhoon_data.to_crs(epsg=32634, inplace=True)
typhoon_data['geometry'] = typhoon_data.geometry.buffer(typhoon_data['class'] * 50000)
typhoon_data.to_crs(epsg=4326, inplace=True)

#formatting datetime
typhoon_data['display_time'] = pd.to_datetime(typhoon_data['display_time'])
typhoon_data['display_time'] = typhoon_data['display_time'].dt.tz_localize(None)

#previewing data
typhoon_data.head(10)

In [ ]:
# @title Fetching language use data for Philippines from language use data platform as geojson.

#initializing dataframe and API call
language_data = gpd.GeoDataFrame()
page_num = 0
base_url = "https://ludp.clearglobal.org/public/location/PHL?aggregation=2&output=geojson&fields=proportion_value,individuals_value_weighted,language_rank,language_name,language_code,location_name,location_code,location_level,dataset_name,url,source,datetime_published,reliability_score&page="

#looping through pages of API response and adding to language_data dataframe
while True:
    url = f"{base_url}{page_num}"
    response = requests.get(url)
    language_geojson = response.json()
    if len(language_geojson['features']) == 0:
        print(f"End at {page_num}")
        break
    language_data_page = gpd.GeoDataFrame.from_features(
        language_geojson['features'])
    page_num += 1
    language_data = pd.concat([language_data_page, language_data])

#setting geometry and coordinate reference system
language_data = language_data.set_geometry('geometry')
language_data = language_data.set_crs(epsg=4326)

#previewing data
language_data.head(10)

In [ ]:
# @title Merging and preparing data

#merging data with spatial join
merged_data = gpd.sjoin(language_data,typhoon_data)
merged_data['language_name'] = merged_data['language_name'].str[:20]
#filtering typhoon data for relevant points
typhoon_data_filter = typhoon_data[typhoon_data.index.isin(
    merged_data.index_right.unique())]

# Get all unique (original) language names for consistent coloring
all_languages = merged_data['language_name'].unique()

# Get a qualitative color scale
colors = px.colors.qualitative.Plotly
# Create a dictionary mapping each language to a color
language_color_map = {lang: colors[i % len(colors)] for i, lang in enumerate(all_languages)}

In [ ]:
# @title Preparing dashboard to visualise data

#initialising dashboard
app = dash.Dash(__name__)

#setting the layout
app.layout = html.Div(
    [
        html.H1("Languages used by people affected by typhoon Ewiniar", style={'backgroundColor': 'white',"border":"10px white solid",'fontFamily': 'sans-serif'}),
        html.Div([
            dcc.Interval(
                id='interval-component',
                interval=2000,
                n_intervals=0,
                disabled=True # Start disabled
            ),
            html.Button('▶', id='play-pause-button', n_clicks=0,style={'width': '40px', 'height': '40px', 'borderRadius': '50%', 'padding': '0', 'textAlign': 'center', 'lineHeight': '40px', 'border': 'none', 'backgroundColor': '#e7e7e7', 'color': 'black'}),

            html.Div([html.Div("Date/Time", style={'margin-bottom': '10px','fontFamily': 'sans-serif','font-size':'0.8em'}),
            dcc.Slider(
                id='time-slider',
                min=0,
                max=len(typhoon_data_filter['display_time'].dt.strftime("%d/%m, %H:%M").unique())-1,
                step=1,
                value=0,  # Initial value
                marks={i: {'label': display_time, 'style': { 'whiteSpace': 'nowrap', 'fontFamily': 'sans-serif'}} for i, display_time in enumerate(
                    typhoon_data_filter['display_time'].dt.strftime("%d/%m, %H:%M").unique())}
            )],style={'backgroundColor': 'white','width': '80vw','margin-left' : '50px','margin-right' : '25px'}),

        ],
            style={'backgroundColor': 'white','width': '95vw','margin-left' : '25px','margin-right' : '25px','display': 'flex'}),
        html.Div(
            [
                dcc.Graph(id='typhoon-graph',
                          style={'width': '50%', 'display': 'inline-block',"border":"10px white solid",'height': '500px'},config={
        'displayModeBar': False
    }),
                dcc.Graph(id='language-graph',
                          style={'width': '50%', 'display': 'inline-block',"border":"10px white solid",'height': '550px'},config={
        'displayModeBar': False
    })
            ],
            style={'display': 'flex'}
        )
    ],style={'backgroundColor':'white'}
)

#setting callbacks to update chart based on slider
@app.callback(
    [Output('typhoon-graph', 'figure'), Output('language-graph', 'figure')],
    [Input('time-slider', 'value'), Input('typhoon-graph', 'clickData')]
)

#function to update chart and map based on slider
def update_graphs(selected_time_index, clickData):

    # get value from slider
    selected_display_time = typhoon_data_filter['display_time'].dt.strftime("%d/%m, %H:%M").unique()[
        selected_time_index]

    # Filter data for the typhoon map
    filtered_typhoon_data = typhoon_data_filter[typhoon_data_filter['display_time'].dt.strftime("%d/%m, %H:%M")
                                                == selected_display_time]

    # Update typhoon map figure
    typhoon_fig = px.choropleth_map(
        filtered_typhoon_data,
        geojson=filtered_typhoon_data.geometry,
        locations=filtered_typhoon_data.index,
        color="class",
        color_continuous_scale="temps",
        color_continuous_midpoint=3.5,
        range_color=[2, 5],
        map_style="carto-positron",
        zoom=5,
        center={"lat": 12.301530162037709, "lon": 123.80141321621963},
        opacity=0.5,
        hover_data=['class', 'display_time']
    )
    typhoon_fig.update_layout(
        margin={"r": 0, "t": 50, "l": 0, "b": 0},
        coloraxis_colorbar_x=-0.15,
        title='Position and class of typhoon Ewiniar at ' + selected_display_time,
        coloraxis_colorbar=dict(
            title='Class'
        )
    )

    if clickData is None:
        # Filter by slider value
        filtered_data = merged_data[merged_data['display_time'].dt.strftime("%d/%m, %H:%M")
                                    == selected_display_time]
        title_text = 'Top 10 languages in districts affected at ' + selected_display_time
    else:
        display_time = clickData['points'][0]['customdata'][1]
        filtered_data = merged_data[merged_data['display_time']
                                    == display_time]
        title_text = 'Top 10 languages in districts affected at ' + display_time + \
            '(' + ' ,'.join(filtered_data['location_name'].unique()) + ')'

    filtered_data = filtered_data.groupby(['language_name']).agg(
        {'individuals_value_weighted': 'sum'}).reset_index()
    filtered_data['rank'] = filtered_data['individuals_value_weighted'].rank(
        ascending=False)
    total = filtered_data['individuals_value_weighted'].sum()
    filtered_data['proportion_population']= (filtered_data['individuals_value_weighted'] / total) * 100

    filtered_data = filtered_data.sort_values(['individuals_value_weighted'], ascending=False)
    filtered_data = filtered_data[filtered_data['rank'] <= 10]

    lang_fig_updated = px.bar(filtered_data, y='language_name', x='proportion_population', color='language_name',
                              labels={'language_name': 'Language', 'proportion_population': 'Population %'},color_discrete_map=language_color_map)
    lang_fig_updated.update_layout(title=title_text,xaxis=dict(range=[0, 100]), margin=dict(l=200),showlegend=False)
    return typhoon_fig, lang_fig_updated


# Callback to toggle the animation
@app.callback(
    Output('interval-component', 'disabled'),
    [Input('play-pause-button', 'n_clicks')],
    [State('interval-component', 'disabled')]
)
def toggle_animation(n_clicks, currently_disabled):
    if n_clicks:
        return not currently_disabled
    return currently_disabled

# Callback to update the slider based on the interval
@app.callback(
    Output('time-slider', 'value'),
    [Input('interval-component', 'n_intervals')],
    [State('time-slider', 'max')],
    prevent_initial_call=True # Prevent the callback from firing on page load
)
def update_slider(n_intervals, max_value):
    # Calculate the next value for the slider, cycling back to 0 at the end
    next_value = n_intervals % (max_value + 1)
    return next_value

# New callback to update button text based on animation state
@app.callback(
    Output('play-pause-button', 'children'),
    [Input('interval-component', 'disabled')]
)
def update_button_text(disabled):
    if disabled:
        return '▶'
    else:
        return '❚❚'


if __name__ == '__main__':
    #app.run(debug=True)  #uncomment to view in notebook
    app.run(jupyter_mode="external")